In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
matches = pd.read_csv(r"C:\Users\Mayank\Downloads\matches (1).csv", index_col=0)

In [3]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [4]:
matches["date"] = pd.to_datetime(matches["date"])

In [5]:
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                     float64
pkatt                  float64
season                   int64
team                    object
dtype: object

In [6]:
matches['team'].value_counts()

team
Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Liverpool                   38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   34
Watford                     33
Norwich City                33
Name: count, dtype: int64

In [7]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

matches["opp_code"] = matches["opponent"].astype("category").cat.codes

matches["team_code"] = matches["team"].astype("category").cat.codes

matches["hour"] = matches["time"].str.replace(":.+","", regex=True).astype("int")

matches["day_code"] = matches["date"].dt.dayofweek

matches["target"] = (matches["result"] == "W").astype("int")

In [8]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opp_code,team_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,0.0,0.0,2022,Manchester City,0,18,12,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,0.0,0.0,2022,Manchester City,1,15,12,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,2022,Manchester City,1,0,12,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,2022,Manchester City,0,10,12,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,0.0,0.0,2022,Manchester City,1,17,12,15,5,0


In [9]:
cols_we_need = ["team","opponent","gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt",'gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling',"venue_code", "opp_code", "hour", "day_code","team_code"]

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 50, min_samples_split = 10, random_state = 1)

train = matches[matches["date"] < '2022-01-01']

test = matches[matches["date"] > '2022-01-01']

predictors = ["venue_code", "opp_code", "hour", "day_code","team_code"]

rf.fit(train[predictors], train["target"])

preds = rf.predict(test[predictors])

from sklearn.metrics import accuracy_score

acc = accuracy_score(test["target"], preds)

acc

comb = pd.DataFrame(dict(actual = test["target"], prediction=preds))

pd.crosstab(index=comb["actual"], columns=comb["prediction"])

prediction,0,1
actual,,
0,147,25
1,79,25


In [11]:
from sklearn.metrics import precision_score

In [12]:
precision_score(test["target"], preds)

0.5

In [13]:
grouped_matches = matches.groupby('team')

In [14]:
group = grouped_matches.get_group('Manchester United')

In [15]:
group.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opp_code,team_code,hour,day_code,target
0,2021-08-14,12:30,Premier League,Matchweek 1,Sat,Home,W,5.0,1.0,Leeds United,...,0.0,0.0,2022,Manchester United,1,9,13,12,5,1
1,2021-08-22,14:00,Premier League,Matchweek 2,Sun,Away,D,1.0,1.0,Southampton,...,0.0,0.0,2022,Manchester United,0,17,13,14,6,0
2,2021-08-29,16:30,Premier League,Matchweek 3,Sun,Away,W,1.0,0.0,Wolves,...,0.0,0.0,2022,Manchester United,0,22,13,16,6,1
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,1.0,Newcastle Utd,...,0.0,0.0,2022,Manchester United,1,14,13,15,5,1
5,2021-09-19,14:00,Premier League,Matchweek 5,Sun,Away,W,2.0,1.0,West Ham,...,0.0,0.0,2022,Manchester United,0,21,13,14,6,1


In [16]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [17]:
new_cols 

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [18]:
def rolling_avgs(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [19]:
rolling_avgs(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,20:00,Premier League,Matchweek 5,Sat,Away,W,4.0,1.0,Newcastle Utd,...,5,1,1.666667,3.666667,9.000000,2.333333,19.800000,0.333333,0.666667,0.666667
7,2020-10-24,17:30,Premier League,Matchweek 6,Sat,Home,D,0.0,0.0,Chelsea,...,5,0,2.666667,3.000000,12.333333,4.666667,20.000000,0.000000,0.666667,1.000000
9,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Home,L,0.0,1.0,Arsenal,...,6,0,1.666667,2.333333,15.000000,5.333333,19.700000,0.000000,0.333333,0.666667
11,2020-11-07,12:30,Premier League,Matchweek 8,Sat,Away,W,3.0,1.0,Everton,...,5,1,1.333333,0.666667,16.666667,5.666667,18.300000,0.000000,0.000000,0.333333
12,2020-11-21,20:00,Premier League,Matchweek 9,Sat,Home,W,1.0,0.0,West Brom,...,5,1,1.000000,0.666667,12.000000,3.666667,18.733333,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2022-04-02,17:30,Premier League,Matchweek 31,Sat,Home,D,1.0,1.0,Leicester City,...,5,0,1.333333,2.000000,12.666667,3.666667,15.333333,0.333333,0.000000,0.000000
41,2022-04-09,12:30,Premier League,Matchweek 32,Sat,Away,L,0.0,1.0,Everton,...,5,0,1.666667,2.333333,9.000000,4.333333,14.333333,0.000000,0.000000,0.000000
42,2022-04-16,15:00,Premier League,Matchweek 33,Sat,Home,W,3.0,2.0,Norwich City,...,5,1,1.333333,1.333333,11.333333,5.000000,15.333333,0.000000,0.000000,0.000000
43,2022-04-19,20:00,Premier League,Matchweek 30,Tue,Away,L,0.0,4.0,Liverpool,...,1,0,1.333333,1.333333,14.333333,6.000000,15.666667,0.333333,0.000000,0.000000


In [20]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_avgs(x, cols, new_cols))

In [21]:
matches_rolling.head()

date   time            comp        round  day venue result  \
team                                                                          
Arsenal 6  2020-10-04  14:00  Premier League  Matchweek 4  Sun  Home      W   
        7  2020-10-17  17:30  Premier League  Matchweek 5  Sat  Away      L   
        9  2020-10-25  19:15  Premier League  Matchweek 6  Sun  Home      L   
        11 2020-11-01  16:30  Premier League  Matchweek 7  Sun  Away      W   
        13 2020-11-08  19:15  Premier League  Matchweek 8  Sun  Home      L   

             gf   ga         opponent  ...  day_code  target  gf_rolling  \
team                                   ...                                 
Arsenal 6   2.0  1.0    Sheffield Utd  ...         6       1    2.000000   
        7   0.0  1.0  Manchester City  ...         5       0    1.666667   
        9   0.0  1.0   Leicester City  ...         6       0    1.000000   
        11  1.0  0.0   Manchester Utd  ...         6       1    0.666667   
        13  0.0  3.0      Aston Villa  ...         6       0    0.333333   

            ga_rolling sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                    
Arsenal 6     1.333333   7.666667    3.666667    14.733333   0.666667   
        7     1.666667   5.333333    3.666667    15.766667   0.000000   
        9     1.666667   7.000000    3.666667    16.733333   0.666667   
        11    1.000000   9.666667    4.000000    16.033333   1.000000   
        13    0.666667   9.666667    2.666667    18.033333   1.000000   

            pk_rolling  pkatt_rolling  
team                                   
Arsenal 6     0.000000       0.000000  
        7     0.000000       0.000000  
        9     0.000000       0.000000  
        11    0.000000       0.000000  
        13    0.333333       0.333333  

[5 rows x 41 columns]

In [22]:
matches_rolling.to_csv('final_df_Pl.csv', index=False)

In [23]:
pl_pred_df = matches_rolling[cols_we_need]

In [24]:
pl_pred_df

team         opponent   gf  \
team                                                                        
Arsenal                 6                   Arsenal    Sheffield Utd  2.0   
                        7                   Arsenal  Manchester City  0.0   
                        9                   Arsenal   Leicester City  0.0   
                        11                  Arsenal   Manchester Utd  1.0   
                        13                  Arsenal      Aston Villa  0.0   
...                                             ...              ...  ...   
Wolverhampton Wanderers 32  Wolverhampton Wanderers          Everton  1.0   
                        33  Wolverhampton Wanderers     Leeds United  2.0   
                        34  Wolverhampton Wanderers      Aston Villa  2.0   
                        35  Wolverhampton Wanderers    Newcastle Utd  0.0   
                        36  Wolverhampton Wanderers          Burnley  0.0   

                             ga    sh  sot  dist   fk   pk  pkatt  ...  \
team                                                               ...   
Arsenal                 6   1.0   6.0  5.0  16.7  0.0  0.0    0.0  ...   
                        7   1.0  11.0  3.0  18.2  2.0  0.0    0.0  ...   
                        9   1.0  12.0  4.0  13.2  1.0  0.0    0.0  ...   
                        11  0.0   6.0  1.0  22.7  0.0  1.0    1.0  ...   
                        13  3.0  13.0  2.0  15.6  0.0  0.0    0.0  ...   
...                         ...   ...  ...   ...  ...  ...    ...  ...   
Wolverhampton Wanderers 32  0.0  14.0  3.0  20.7  0.0  0.0    0.0  ...   
                        33  3.0  14.0  8.0  20.6  0.0  0.0    0.0  ...   
                        34  1.0  11.0  4.0  14.3  0.0  0.0    0.0  ...   
                        35  1.0   5.0  2.0  18.0  0.0  0.0    0.0  ...   
                        36  1.0  10.0  4.0  18.6  0.0  0.0    0.0  ...   

                            sot_rolling  dist_rolling  fk_rolling  pk_rolling  \
team                                                                            
Arsenal                 6      3.666667     14.733333    0.666667    0.000000   
                        7      3.666667     15.766667    0.000000    0.000000   
                        9      3.666667     16.733333    0.666667    0.000000   
                        11     4.000000     16.033333    1.000000    0.000000   
                        13     2.666667     18.033333    1.000000    0.333333   
...                                 ...           ...         ...         ...   
Wolverhampton Wanderers 32     3.666667     19.300000    0.000000    0.000000   
                        33     4.333333     19.600000    0.000000    0.000000   
                        34     5.333333     19.833333    0.000000    0.000000   
                        35     5.000000     18.533333    0.000000    0.000000   
                        36     4.666667     17.633333    0.000000    0.000000   

                            pkatt_rolling  venue_code  opp_code  hour  \
team                                                                    
Arsenal                 6        0.000000           1        16    14   
                        7        0.000000           0        12    17   
                        9        0.000000           1        10    19   
                        11       0.000000           0        13    16   
                        13       0.333333           1         1    19   
...                                   ...         ...       ...   ...   
Wolverhampton Wanderers 32       0.000000           0         7    14   
                        33       0.000000           1         9    20   
                        34       0.000000           1         1    15   
                        35       0.000000           0        14    20   
                        36       0.000000           0         4    14   

                            day_code  team_code  
team                        

In [25]:
pl_pred_df.index = range(pl_pred_df.shape[0])

In [26]:
pl_pred_df.head()

,team,opponent,gf,ga,sh,sot,dist,fk,pk,pkatt,...,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,venue_code,opp_code,hour,day_code,team_code
0,Arsenal,Sheffield Utd,2.0,1.0,6.0,5.0,16.7,0.0,0.0,0.0,...,3.666667,14.733333,0.666667,0.000000,0.000000,1,16,14,6,0
1,Arsenal,Manchester City,0.0,1.0,11.0,3.0,18.2,2.0,0.0,0.0,...,3.666667,15.766667,0.000000,0.000000,0.000000,0,12,17,5,0
2,Arsenal,Leicester City,0.0,1.0,12.0,4.0,13.2,1.0,0.0,0.0,...,3.666667,16.733333,0.666667,0.000000,0.000000,1,10,19,6,0
3,Arsenal,Manchester Utd,1.0,0.0,6.0,1.0,22.7,0.0,1.0,1.0,...,4.000000,16.033333,1.000000,0.000000,0.000000,0,13,16,6,0
4,Arsenal,Aston Villa,0.0,3.0,13.0,2.0,15.6,0.0,0.0,0.0,...,2.666667,18.033333,1.000000,0.333333,0.333333,1,1,19,6,0


In [27]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    comb = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return comb, precision

In [28]:
comb, precision = make_predictions(matches_rolling, predictors + new_cols)

In [29]:
precision

0.5833333333333334

In [30]:
comb

actual  predicted
team                                         
Arsenal                 27       0          0
                        28       1          1
                        29       1          0
                        30       1          1
                        31       1          1
...                            ...        ...
Wolverhampton Wanderers 32       1          0
                        33       0          0
                        34       1          0
                        35       0          0
                        36       0          0

[276 rows x 2 columns]

In [31]:
acc = accuracy_score(test["target"], preds)
acc

0.6231884057971014

In [32]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

def make_prediction(model, match_data):

    match_df = pd.DataFrame([match_data])

    prediction = model.predict(match_df[predictors + new_cols])[0]

    return prediction

match_data = {'team': 'Arsenal', 'opponent': 'Chelsea', 'gf': 2, 'ga': 1, 'sh': 10, 'sot': 5, 'dist': 30,
              'fk': 5, 'pk': 1, 'pkatt': 2, 'gf_rolling': 2, 'ga_rolling': 1, 'sh_rolling': 8,
              'sot_rolling': 4, 'dist_rolling': 25, 'fk_rolling': 4, 'pk_rolling': 1, 'pkatt_rolling': 2,
              'venue_code': 1, 'opp_code': 2, 'hour': 15, 'day_code': 3, 'team_code': 10}

predicted_winner = make_prediction(rf, match_data)

if predicted_winner == 0:
    print(f"The predicted winner is: {match_data['opponent']}")
else:
    print(f"The predicted winner is: {match_data['team']}")


The predicted winner is: Chelsea


In [33]:
import pickle

In [34]:
filename = 'trained_model.sav'

In [35]:
pickle.dump(rf, open(filename,'wb'))

In [36]:
load_model = pickle.load(open(filename, 'rb'))

In [37]:
def make_prediction(model, match_data):

    match_df = pd.DataFrame([match_data])

    prediction = model.predict(match_df[predictors + new_cols])[0]

    return prediction

match_data = {'team': 'Arsenal', 'opponent': 'Chelsea', 'gf': 2, 'ga': 1, 'sh': 10, 'sot': 5, 'dist': 30,
              'fk': 5, 'pk': 1, 'pkatt': 2, 'gf_rolling': 2, 'ga_rolling': 1, 'sh_rolling': 8,
              'sot_rolling': 4, 'dist_rolling': 25, 'fk_rolling': 4, 'pk_rolling': 1, 'pkatt_rolling': 2,
              'venue_code': 1, 'opp_code': 2, 'hour': 15, 'day_code': 3, 'team_code': 10}

predicted_winner = make_prediction(load_model, match_data)

if predicted_winner == 0:
    print(f"The predicted winner is: {match_data['opponent']}")
else:
    print(f"The predicted winner is: {match_data['team']}")

The predicted winner is: Chelsea
